# diabetes prediction using embeddings

In [74]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import pandas as pd

In [106]:
data=pd.read_csv('datasets/automobile.csv').dropna()

In [110]:
data.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.4,10.0,102.0,5500.0,24,30,13950
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.4,8.0,115.0,5500.0,18,22,17450
6,1,158.0,audi,gas,std,four,sedan,fwd,front,105.8,...,136,mpfi,3.19,3.4,8.5,110.0,5500.0,19,25,17710
8,1,158.0,audi,gas,turbo,four,sedan,fwd,front,105.8,...,131,mpfi,3.13,3.4,8.3,140.0,5500.0,17,20,23875
9,2,192.0,bmw,gas,std,two,sedan,rwd,front,101.2,...,108,mpfi,3.50,2.8,8.8,101.0,5800.0,23,29,16430


In [112]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159 entries, 3 to 200
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          159 non-null    int64  
 1   normalized-losses  159 non-null    float64
 2   make               159 non-null    object 
 3   fuel-type          159 non-null    object 
 4   aspiration         159 non-null    object 
 5   num-of-doors       159 non-null    object 
 6   body-style         159 non-null    object 
 7   drive-wheels       159 non-null    object 
 8   engine-location    159 non-null    object 
 9   wheel-base         159 non-null    float64
 10  length             159 non-null    float64
 11  width              159 non-null    float64
 12  height             159 non-null    float64
 13  curb-weight        159 non-null    int64  
 14  engine-type        159 non-null    object 
 15  num-of-cylinders   159 non-null    object 
 16  engine-size        159 non-null

In [137]:
data.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
3,4,164.0,audi,1,0,0,3,1,front,99.8,...,109,4,3.19,3.4,10.0,102.0,5500.0,24,30,13950
4,4,164.0,audi,1,0,0,3,0,front,99.4,...,136,4,3.19,3.4,8.0,115.0,5500.0,18,22,17450
6,3,158.0,audi,1,0,0,3,1,front,105.8,...,136,4,3.19,3.4,8.5,110.0,5500.0,19,25,17710
8,3,158.0,audi,1,1,0,3,1,front,105.8,...,131,4,3.13,3.4,8.3,140.0,5500.0,17,20,23875
9,4,192.0,bmw,1,0,1,3,2,front,101.2,...,108,4,3.50,2.8,8.8,101.0,5800.0,23,29,16430


In [147]:
# data.drop(columns=['engine-location'],inplace=True)
data

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,wheel-base,length,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
3,4,164.0,audi,1,0,0,3,1,99.8,176.6,...,109,4,3.19,3.40,10.0,102.0,5500.0,24,30,13950
4,4,164.0,audi,1,0,0,3,0,99.4,176.6,...,136,4,3.19,3.40,8.0,115.0,5500.0,18,22,17450
6,3,158.0,audi,1,0,0,3,1,105.8,192.7,...,136,4,3.19,3.40,8.5,110.0,5500.0,19,25,17710
8,3,158.0,audi,1,1,0,3,1,105.8,192.7,...,131,4,3.13,3.40,8.3,140.0,5500.0,17,20,23875
9,4,192.0,bmw,1,0,1,3,2,101.2,176.8,...,108,4,3.50,2.80,8.8,101.0,5800.0,23,29,16430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,1,95.0,volvo,1,0,0,3,2,109.1,188.8,...,141,4,3.78,3.15,9.5,114.0,5400.0,23,28,16845
197,1,95.0,volvo,1,1,0,3,2,109.1,188.8,...,141,4,3.78,3.15,8.7,160.0,5300.0,19,25,19045
198,1,95.0,volvo,1,0,0,3,2,109.1,188.8,...,173,4,3.58,2.87,8.8,134.0,5500.0,18,23,21485
199,1,95.0,volvo,0,1,0,3,2,109.1,188.8,...,145,2,3.01,3.40,23.0,106.0,4800.0,26,27,22470


In [149]:
categorical_features=['symboling','fuel-type','aspiration','num-of-doors','body-style','drive-wheels','engine-type','num-of-cylinders','fuel-system']
out_feature=['price']

# categorical features -- embedding layers

In [152]:
from sklearn.preprocessing import LabelEncoder
lbl_encoder={}
for feature in categorical_features:
    lbl_encoder[feature]=LabelEncoder()
    data[feature]=lbl_encoder[feature].fit_transform(data[feature])


In [156]:
for i in data.columns:
    print(f'column name {i} and unique value are {len(data[i].unique())}')

column name symboling and unique value are 6
column name normalized-losses and unique value are 51
column name make and unique value are 18
column name fuel-type and unique value are 2
column name aspiration and unique value are 2
column name num-of-doors and unique value are 2
column name body-style and unique value are 5
column name drive-wheels and unique value are 3
column name wheel-base and unique value are 40
column name length and unique value are 56
column name width and unique value are 33
column name height and unique value are 39
column name curb-weight and unique value are 136
column name engine-type and unique value are 5
column name num-of-cylinders and unique value are 5
column name engine-size and unique value are 32
column name fuel-system and unique value are 6
column name bore and unique value are 33
column name stroke and unique value are 31
column name compression-ratio and unique value are 29
column name horsepower and unique value are 48
column name peak-rpm and

In [176]:
import numpy as np
cat_features=np.stack([data[i] for i in categorical_features])
cat_features

array([[4, 4, 3, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 1, 1],
       ...,
       [2, 2, 2, ..., 4, 2, 2],
       [2, 1, 1, ..., 3, 3, 2],
       [4, 4, 4, ..., 4, 2, 4]], dtype=int64)

In [180]:
import torch
cat_features=torch.tensor(cat_features,dtype=torch.int64)
cat_features

tensor([[4, 4, 3,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 0, 1],
        [0, 0, 0,  ..., 0, 1, 1],
        ...,
        [2, 2, 2,  ..., 4, 2, 2],
        [2, 1, 1,  ..., 3, 3, 2],
        [4, 4, 4,  ..., 4, 2, 4]])

# categorical features -- embedding layers 

In [195]:
continous_features=[]
for i in data.columns:
    if i in categorical_features or i == 'price':
        pass
    else:
        continous_features.append(i)
# out_features=data['price']
continous_features

['normalized-losses',
 'make',
 'wheel-base',
 'length',
 'width',
 'height',
 'curb-weight',
 'engine-size',
 'bore',
 'stroke',
 'compression-ratio',
 'horsepower',
 'peak-rpm',
 'city-mpg',
 'highway-mpg']

In [197]:
## stacking continous variables to tensor

In [201]:
cont_values=np.stack([data[i].values for i in continous_features],axis=1)
cont_values=torch.tensor(cont_values,dtype=torch.float)
cont_values

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [199]:
# from sklearn.model_selection import train_test_split
# xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=1)
# xtrain.shape,xtest.shape,ytest.shape,ytrain.shape